In [88]:
from utils.df_handle import *

In [146]:
sql = \
"""
SELECT DISTINCT masp, kenh from d_forecast_sc
"""

In [147]:
all_ver = get_ps_df(sql)

In [148]:
all_ver.head()

,masp,kenh
0,EH123,OTC
1,EH065,INS
2,OH052,CLC
3,EH086,CLC
4,EH117,CLC


In [149]:
sql = \
"""
SELECT DISTINCT masp, kenh from d_forecast_sc where version = 0
"""

In [150]:
ver0 = get_ps_df(sql)

In [151]:
all_ver.shape

(264, 2)

In [152]:
ver0.shape

(256, 2)

In [153]:
ver0['count'] = 1

In [154]:
all_ver = all_ver.merge(ver0, on=['masp','kenh'], how='left')

In [155]:
all_ver.to_clipboard()

In [407]:
v1=1
v2=2
sql = \
f"""
SELECT masp, month, kenh, version, sum(fcvalues) as fcvalues from d_forecast_sc WHERE version in ({v1},{v2})
GROUP BY 1,2,3,4
"""
df = get_ps_df(sql)
df.head()
dk1 = df.version==v1
dk2 = df.version==v2
df1=df[dk1].copy()
df2=df[dk2].copy()
# drop_cols(df1, ['version','fcvalues'])
# drop_cols(df2, ['version','fcvalues'])
# df1 = dropdup(df1,1)
# df2 = dropdup(df2,1)
# dfs_diff(df1,df2).to_clipboard()

# df3.to_clipboard()

In [388]:
df3 = df2.merge(df1, on=['masp','month','kenh'], how='left', suffixes=['','_lv'])

In [389]:
df3.fcvalues_lv.fillna(0, inplace=True)

In [390]:
df3['gap'] = df3.fcvalues - df3.fcvalues_lv

In [391]:
df3['gap'].sum()

-3000.0

In [392]:
# df1.fcvalues.sum() - df2.fcvalues.sum()

In [393]:
# df1.shape

In [394]:
# df2.shape

In [395]:
df3.to_clipboard()

In [396]:
dk3 = df3['gap'] >= 1
dk4 = df3['gap'] <= -1
df4 = df3[dk3 | dk4].copy()
lst = ['masp', 'month', 'kenh', 'version', 'fcvalues','fcvalues_lv','gap']
df5 = df4[lst].copy()
execute_values_insert(df5,"d_forecast_sc_gap")

In [488]:
# Check Nguoc

In [500]:
v1=1
v2=2
sql = \
f"""
SELECT masp, month, kenh, version, sum(fcvalues) as fcvalues from d_forecast_sc WHERE version in ({v1},{v2})
GROUP BY 1,2,3,4
"""
df = get_ps_df(sql)
df.head()
dk1 = df.version==v1
dk2 = df.version==v2
df1=df[dk1].copy()
df2=df[dk2].copy()

In [501]:
df6 = df1.merge(df2, on=['masp','month','kenh'], how='left', suffixes=['_lv',''])

In [502]:
dk1 = df6.version.isna()
df6 = df6[dk1].copy()

In [503]:
df6.version.fillna(int(v2),inplace=True)

In [504]:
df6.fcvalues.fillna(0, inplace=True)

In [505]:
df6['gap'] = df6.fcvalues - df6.fcvalues_lv

In [506]:
dk3 = df6['gap'] >= 1
dk4 = df6['gap'] <= -1
df7 = df6[dk3 | dk4].copy()

In [508]:
df7.to_csv("v1NotInV2.csv", index=False)

In [497]:
lst = ['masp', 'month', 'kenh', 'version', 'fcvalues','fcvalues_lv','gap']
df8 = df7[lst].copy()
execute_values_insert(df8,"d_forecast_sc_gap")

Data inserted using execute_values() successfully..


In [510]:
# df8.shape